In [1]:
import os
import requests
import json
import time
import pandas as pd
from copy import deepcopy
from itertools import combinations
from pprint import pp

### Verify connection to the API

In [2]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']
headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

Response from Status: 200
{"message":"Connection to DNV WindFarmer Services API was successful.","windFarmerServicesAPIVersion":"2.6.14","calculationLibraryVersion":"2.6.10.0"}


#  **Part 1** - generating a demo batch of API inputs [OPTIONAL, skip if you have generated the batch otherwise] 

Let's imagine we are asked to remove three turbines from a layout in the most energy efficient way. We'll do that by identifying the worst 10 turbines in the layout and later evaulating the yield of a batch of alternative layouts, each created by removing 3 different turbines from the set of 10 worst producing locations.  

Initial calculation to get a feel where the most heavily waked zones are 

In [3]:
# load the input json for The Bowl
with open('../../../DemoData/TheBowl/TheBowl.json', 'r') as f:
    json_string = f.read()
    input_data = json.loads(json_string)
    
start = time.time()
response = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data)
    
print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response.status_code != 200:
    print(json.loads(response.content)['detail'])

Response 200 - OK in 4.62s


identifting the worst 10 turbines

In [4]:
result = json.loads(response.content)
yields = [x['fullAnnualYield_MWh_per_year'] for x in result['windFarmAepOutputs'][0]['turbineResults']]
labels = [x['turbineName'] for x in result['windFarmAepOutputs'][0]['turbineResults']]
yields_df = pd.DataFrame.from_dict({k:v for k,v in zip(labels, yields)}, orient='index', columns=['Yield (MWh/year)'])
yields_df.sort_values(by='Yield (MWh/year)', ascending=True, inplace=True)
worst_10_turbines = list(yields_df.iloc[:10].index)
print(worst_10_turbines)

['T35', 'T42', 'T55', 'T52', 'T47', 'T23', 'T60', 'T30', 'T28', 'T29']


generating all the possible combinations of removing two out of ten candidate turbines

In [5]:
turbine_removal_alternatives = list(combinations(worst_10_turbines, 3))
print(turbine_removal_alternatives)

[('T35', 'T42', 'T55'), ('T35', 'T42', 'T52'), ('T35', 'T42', 'T47'), ('T35', 'T42', 'T23'), ('T35', 'T42', 'T60'), ('T35', 'T42', 'T30'), ('T35', 'T42', 'T28'), ('T35', 'T42', 'T29'), ('T35', 'T55', 'T52'), ('T35', 'T55', 'T47'), ('T35', 'T55', 'T23'), ('T35', 'T55', 'T60'), ('T35', 'T55', 'T30'), ('T35', 'T55', 'T28'), ('T35', 'T55', 'T29'), ('T35', 'T52', 'T47'), ('T35', 'T52', 'T23'), ('T35', 'T52', 'T60'), ('T35', 'T52', 'T30'), ('T35', 'T52', 'T28'), ('T35', 'T52', 'T29'), ('T35', 'T47', 'T23'), ('T35', 'T47', 'T60'), ('T35', 'T47', 'T30'), ('T35', 'T47', 'T28'), ('T35', 'T47', 'T29'), ('T35', 'T23', 'T60'), ('T35', 'T23', 'T30'), ('T35', 'T23', 'T28'), ('T35', 'T23', 'T29'), ('T35', 'T60', 'T30'), ('T35', 'T60', 'T28'), ('T35', 'T60', 'T29'), ('T35', 'T30', 'T28'), ('T35', 'T30', 'T29'), ('T35', 'T28', 'T29'), ('T42', 'T55', 'T52'), ('T42', 'T55', 'T47'), ('T42', 'T55', 'T23'), ('T42', 'T55', 'T60'), ('T42', 'T55', 'T30'), ('T42', 'T55', 'T28'), ('T42', 'T55', 'T29'), ('T42', 'T

generate API inputs for each alternative

In [6]:
for alternative in turbine_removal_alternatives:
    temp_json = deepcopy(input_data)
    for turbine in input_data["windFarms"][0]["turbines"]:
        if turbine["name"] in alternative:
            temp_json["windFarms"][0]["turbines"].remove(turbine)
    if not os.path.exists('./InputBatch'):
        os.makedirs('./InputBatch')
    with open('./InputBatch/{0}.json'.format('_'.join(alternative)), 'w') as f:
        f.write(json.dumps(temp_json, indent=4))

#  **Part 2** - batch processing [if you're using this for API inputs generated outside of the script, put them in a folder `InputBatch`, next to the script]

### **Define user inputs**

In [7]:
# specify the desired model combination
WAKE_MODEL_CHOICE = "EddyViscosity" # EddyViscosity/ModifiedPark/TurbOPark/CFDML
BLOCKAGE_MODEL_CHOICE = "BEET" # BEET/CFDML
CALCULATE_EFFICIENCIES = True #True/False 
  
# specify the path to the input json files
PATH_TO_INPUTS = './InputBatch'

**Note**: the part responsible for model settings in the input jsons contained in the `./InputBatch` directory will get overwritten by the settings specified in the code cell above  

In [8]:
# the below dictionaries contain various settings, of which we make a selection for the particular run  (inside set_model_settings)
lwf_paramters = { # LWF's default offshore settings
                    "baseRoughnessZ01": 0.0002,
                    "increasedRoughnessZ02": 0.0192,
                    "geometricWidthDiameters": 1.0,
                    "recoveryStartDiameters": 120.0,
                    "fiftyPercentRecoveryDiameters": 40.0
                }
wake_models = {"EddyViscosity": {"model_key":"eddyViscosity", "model_settings":{"useLargeWindFarmModel": True, "largeWindFarmCorrectionParameters": lwf_paramters}},
               "ModifiedPark":  {"model_key":"modifiedPark", "model_settings": {"useLargeWindFarmModel": True, "largeWindFarmCorrectionParameters": lwf_paramters}},
               "TurbOPark": {"model_key":"turbOPark", "model_settings": {"wakeExpansion": 0.04}},
               "CFDML": {"model_key":"cfdml", "model_settings":{"gnnType": "Offshore", "gnnStabilityClass": "UnstableNeutral"}}}
blockage_models = {"BEET": {"model_key":"beet", "model_settings":{"significantAtmosphericStability": False,
                                                                  "inclusionOfNeighborsBufferZoneInMeters": 1000.0,
                                                                  "blockageCorrectionApplicationMethod": "OnWindSpeed"}},
                   "CFDML": {"model_key":"cfdml", "model_settings":{"cfdmlSettings": {"gnnType": "Offshore", "gnnStabilityClass": "UnstableNeutral"}, 
                                                                    "blockageCorrectionApplicationMethod": "OnWindSpeed",
                                                                    "cfdmlBlockageWindSpeedDependency": "FromBlockageExtrapolationCurve"}}}
# the below functions wrap steps used more than once in later code-cells
def switch_off_fpm_export(input_json):
    # this function makes sure fpm export is switched off - to speed up API response time
    for fpm in input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"].keys():
        if fpm != "localTurbineWindSpeedsOutputSettings":
            input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"][fpm] = False
        else:
            input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"][fpm] = None

def set_model_settings(input_json):
    # let's set the modeling options contained in the json
    input_json["energyEfficienciesSettings"]["calculateEfficiencies"] = CALCULATE_EFFICIENCIES
    # pull the relevant default settings from the dicts predefined above
    input_json["energyEfficienciesSettings"]["wakeModel"]["wakeModelType"] = WAKE_MODEL_CHOICE
    input_json["energyEfficienciesSettings"]["wakeModel"][wake_models[WAKE_MODEL_CHOICE]["model_key"]] = wake_models[WAKE_MODEL_CHOICE]["model_settings"]
    input_json["energyEfficienciesSettings"]["blockageModel"]["blockageModelType"] = BLOCKAGE_MODEL_CHOICE if WAKE_MODEL_CHOICE != "CFDML" else "CFDML"
    input_json["energyEfficienciesSettings"]["blockageModel"][blockage_models[BLOCKAGE_MODEL_CHOICE]["model_key"]] = blockage_models[BLOCKAGE_MODEL_CHOICE]["model_settings"]
    switch_off_fpm_export(input_json)

def submit_the_job(input_json_path, job_id_dict):
    # load the json inputs 
    with open(input_json_path) as f:
        json_string = f.read()
        input_json = json.loads(json_string)
    # override the modeling settings pre-exisitng in the json with the ones specified above
    set_model_settings(input_json)
    # save the updated json
    with open(input_json_path, 'w') as f:
        f.write(json.dumps(input_json, indent=4))
    start = time.time()
    response = requests.post(
        api_url + 'AnnualEnergyProductionAsync', 
        headers=headers,
        json = input_json
    )
    print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
    response_json = json.loads(response.content)
    if response.status_code == 202:
        job_id_dict[os.path.basename(input_json_path)] = response_json['jobId']
        print("job ID: " + str(response_json['jobId']))
    elif response.status_code == 400:
        if (detail := response_json.get('detail')):
            print(f"Bad request: {detail}")
        if (errors := response_json.get('errors')):
            print("Errors:")
            pp(errors)
    return job_id_dict

def poll_results(job_id_dict):
    if not os.path.exists(f'./Results/'):
        os.mkdir(f'./Results/')
    jobs_completed_dict = {k:False for k in job_id_dict.keys()}
    jobs_failed_list = []
    while not all(jobs_completed_dict.values()):
        for job_filename, job_id in job_id_dict.items():
            if not jobs_completed_dict[job_filename]:
                response = requests.get(api_url + 'AnnualEnergyProductionAsync', headers = headers, params={'jobId':job_id})
                response_json = json.loads(response.content)
                if response_json['status'] == 'SUCCESS':
                    jobs_completed_dict[job_filename] = True
                    print(f'Job {job_filename} is completed')
                    with open(f'./Results/{job_filename}', 'w') as f:
                        f.write(json.dumps(response_json, indent=4))
                elif response_json['status'] == 'FAILED':
                    error_message = response_json.get('message', 'Unknown error')
                    print(f'Job {job_id} has FAILED: {error_message}')
                    jobs_failed_list.append(job_filename)
                    jobs_completed_dict[job_filename] = True
                else:
                    message = response_json.get('message')
                    stage_message = response_json.get('stageMessage')
                    progress = str(response_json['progress']) + '%' if 'progress' in response_json else None
                    progress_message = ' - '.join(x for x in [progress, message, stage_message] if x)
                    print(f'Job {job_filename} is {response_json["status"]}, JobID = {job_id}', {progress_message})
        print("No. of completed jobs: " + str(sum(jobs_completed_dict.values())) + " out of " + str(len(jobs_completed_dict)))
        print("Among the completed jobs {} failed.".format(len(jobs_failed_list)))
        time.sleep(5)
    return jobs_completed_dict, jobs_failed_list

submit the batch of all simulation cases to the async endpoint of the API

In [9]:
# submit the jobs
job_id_dict = {}
for input_file in os.listdir(PATH_TO_INPUTS):
    job_id_dict = submit_the_job(os.path.join(PATH_TO_INPUTS,input_file),job_id_dict)

# persist the job_ids to disc
with open('job_id_dict.json', 'w') as f:
    f.write(json.dumps(job_id_dict, indent=4))

Response 202 - Accepted in 1.95s
job ID: 1e795cdf-4902-4296-85c4-57c57627d736
Response 202 - Accepted in 1.19s
job ID: 07e19da4-06c2-4567-b587-2f941376b6af
Response 202 - Accepted in 1.39s
job ID: 5381de68-5131-488d-a6f0-2f7df3c85ceb
Response 202 - Accepted in 1.22s
job ID: dce418d4-e0de-454d-bdb5-368482d259c5
Response 202 - Accepted in 1.30s
job ID: fbad62ff-accb-4657-aaed-fd202e9a7714
Response 202 - Accepted in 1.87s
job ID: 2b3343b7-9f39-4570-81cb-f306bfcde747
Response 202 - Accepted in 1.42s
job ID: c282548a-3611-4643-917a-855b86b899d4
Response 202 - Accepted in 1.16s
job ID: 7629568a-8644-4184-8f11-c4dc444f6e36
Response 202 - Accepted in 1.17s
job ID: 583de7cc-cfc2-4d80-aae7-e9a9623fcefb
Response 202 - Accepted in 1.32s
job ID: 697a220a-afe9-4ff9-93ed-dc702731b52d
Response 202 - Accepted in 1.33s
job ID: 0547cf91-e80d-4de0-9014-e59a836c3ab8
Response 202 - Accepted in 1.03s
job ID: d3ee671b-570a-49c7-a60d-9e3933fca5ec
Response 202 - Accepted in 1.07s
job ID: 99c2421e-514b-4440-b172

## At this point you can go get a coffee or lunch, or even switch the PC off. 

WindFarmerServices API will compute and store your results for 7 days. The subsequent cells read the job id's from file so execution of the notebook may be resumed at any point in time.

Poll the API to see if all results are available...

In [10]:
# load the job ids submitted
with open('job_id_dict.json', 'r') as f:
    job_id_dict = json.load(f)
# poll the API for results, until all jobs are completed
jobs_completed_dict, jobs_failed_list = poll_results(job_id_dict)

Job T23_T28_T29.json is completed
Job T23_T30_T28.json is completed
Job T23_T30_T29.json is PENDING, JobID = 5381de68-5131-488d-a6f0-2f7df3c85ceb {'0.0%'}
Job T23_T60_T28.json is PENDING, JobID = dce418d4-e0de-454d-bdb5-368482d259c5 {'0.0%'}
Job T23_T60_T29.json is completed
Job T23_T60_T30.json is PENDING, JobID = 2b3343b7-9f39-4570-81cb-f306bfcde747 {'0.0%'}
Job T30_T28_T29.json is completed
Job T35_T23_T28.json is completed
Job T35_T23_T29.json is PENDING, JobID = 583de7cc-cfc2-4d80-aae7-e9a9623fcefb {'0.0%'}
Job T35_T23_T30.json is completed
Job T35_T23_T60.json is PENDING, JobID = 0547cf91-e80d-4de0-9014-e59a836c3ab8 {'0.0%'}
Job T35_T28_T29.json is PENDING, JobID = d3ee671b-570a-49c7-a60d-9e3933fca5ec {'0.0%'}
Job T35_T30_T28.json is PENDING, JobID = 99c2421e-514b-4440-b172-00c793ef28af {'0.0%'}
Job T35_T30_T29.json is PENDING, JobID = 04c2a74e-4d6f-4c05-b1a0-00dabbb96602 {'0.0%'}
Job T35_T42_T23.json is PENDING, JobID = 2b48d748-8a94-449e-946b-9165d5302179 {'0.0%'}
Job T35_T42_T

In case some jobs failed (this may happen and not necessarily be linked with your inputs), let's re-submit jobs that did fail...

In [11]:
if len(jobs_failed_list) == 0:
    print("All jobs completed successfully, nothing to re-submit.")
# re-submit failed jobs once
resubmitted_job_id_dict = {}
for job_file in jobs_failed_list:
    print(f'Resubmitting job {job_file}')
    resubmitted_job_id_dict = submit_the_job(os.path.join(PATH_TO_INPUTS,job_file), resubmitted_job_id_dict)
# persist the job_ids to disc
with open('resubmitted_job_id_dict.json', 'w') as f:
    f.write(json.dumps(resubmitted_job_id_dict, indent=4))
# and you can go get coffee again if needed...   

All jobs completed successfully, nothing to re-submit.


... and poll for the re-submitted results

In [12]:
if len(jobs_failed_list) == 0:
    print("No jobs re-submitted, so nothing to poll for.")
# load the job ids submitted
with open('resubmitted_job_id_dict.json', 'r') as f:
    resubmitted_job_id_dict = json.load(f)
# poll the API for results, until all jobs are completed
jobs_completed_dict, jobs_failed_list = poll_results(resubmitted_job_id_dict)

No jobs re-submitted, so nothing to poll for.


# **Part 3** Process the resulting jsons and extract the relevant numbers into a `DataFrame`

In [13]:
case_summary=[]
for results_file in os.listdir('./Results'):
    with open('./Results/'+results_file) as f:
        json_string = f.read()
        results_json = json.loads(json_string)
    subject_farm_results_dict = results_json["results"]["windFarmAepOutputs"][0]
    if CALCULATE_EFFICIENCIES:
        case_summary.append({
            "Case name": results_file,
            "Gross Yield [MWh/Annum]": subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Blockage efficiency [%]": subject_farm_results_dict["blockageOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Internal wake efficiency [%]": subject_farm_results_dict["internalWakesOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["blockageOnAnnualEnergyYield_MWh_per_year"],
            "Hysteresis efficiency [%]": subject_farm_results_dict["hysteresisAdjustmentOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["internalWakesOnAnnualEnergyYield_MWh_per_year"],
            "Internal LWF efficiency [%]": subject_farm_results_dict["largeWindFarmCorrectionOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["hysteresisAdjustmentOnAnnualEnergyYield_MWh_per_year"],
            "External wake efficiency [%]": subject_farm_results_dict["neighborsWakesOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["largeWindFarmCorrectionOnAnnualEnergyYield_MWh_per_year"],
            "Full Yield [MWh/Annum]": subject_farm_results_dict["fullAnnualEnergyYield_MWh_per_year"],
        })
    else:
        case_summary.append({
            "Case name": results_file,
            "Gross Yield [MWh/Annum]": subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Full Yield [MWh/Annum]": subject_farm_results_dict["fullAnnualEnergyYield_MWh_per_year"],
        })
case_summary_df = pd.DataFrame.from_dict(case_summary)
case_summary_df.sort_values(by='Full Yield [MWh/Annum]', ascending=True, inplace=True)
case_summary_df['Yield uplift relative to worst choice [%]'] = round(case_summary_df['Full Yield [MWh/Annum]'] / case_summary_df['Full Yield [MWh/Annum]'].min() * 100 - 100,3)
case_summary_df

,Case name,Gross Yield [MWh/Annum],Blockage efficiency [%],Internal wake efficiency [%],Hysteresis efficiency [%],Internal LWF efficiency [%],External wake efficiency [%],Full Yield [MWh/Annum],Yield uplift relative to worst choice [%]
105,T55_T47_T29.json,1.568098e+06,0.984364,0.923035,1.0,0.942718,1.0,1.343164e+06,0.000
65,T42_T55_T47.json,1.568098e+06,0.984364,0.922956,1.0,0.942801,1.0,1.343166e+06,0.000
75,T47_T28_T29.json,1.568098e+06,0.984364,0.923124,1.0,0.942674,1.0,1.343230e+06,0.005
107,T55_T47_T60.json,1.568098e+06,0.984364,0.922913,1.0,0.942890,1.0,1.343231e+06,0.005
52,T42_T47_T29.json,1.568098e+06,0.984364,0.923115,1.0,0.942684,1.0,1.343231e+06,0.005
...,...,...,...,...,...,...,...,...,...
88,T52_T47_T23.json,1.568098e+06,0.984364,0.923148,1.0,0.943034,1.0,1.343777e+06,0.046
84,T52_T23_T60.json,1.568098e+06,0.984364,0.923026,1.0,0.943162,1.0,1.343783e+06,0.046
108,T55_T52_T23.json,1.568098e+06,0.984364,0.923098,1.0,0.943113,1.0,1.343818e+06,0.049
81,T52_T23_T28.json,1.568098e+06,0.984364,0.923117,1.0,0.943098,1.0,1.343824e+06,0.049


In [14]:
print("best choice -> remove turbines: " + case_summary_df.iloc[-1]['Case name'].replace('.json','').replace('_',','))

best choice -> remove turbines: T35,T52,T23


How to read further values from the results can be determined from the API documentation, or look at the JSON files in your results folder.

https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/WebAPI/ReleaseNotes/releaseNotes.html#openapi-specifications 